In [2]:
import os

In [9]:
if not os.path.exists('data'):
   os.makedirs('data')
   os.makedirs('data/raw')
   print('data directory created successfully')

data directory created successfully


### Importing necessary files

In [3]:
from datetime import datetime, timedelta
import threading
import time
import urllib.request
import os
import pandas as pd

### Functions to generate Start-End dates

In [11]:
def generate_dates_in_start_end_date(input_start_dt, input_end_dt):
    # parse input dates
    start_dt = datetime.strptime(input_start_dt, '%Y%m%d').date()
    end_dt = datetime.strptime(input_end_dt, '%Y%m%d').date()

    # generate list of dates between start_dt and end_dt
    dates = [d.strftime('%Y%m%d') for d in (start_dt + timedelta(n) for n in range((end_dt - start_dt).days + 1))]

    return dates

### Function to append zero to <10 values

In [12]:
def get_zero_appended(value: int)-> str:
    if value < 10:
        return f'0{value}'
    else:
        return str(value)

### Function to download file

In [13]:
def download_file(url, directory, filename):
    urllib.request.urlretrieve(url, os.path.join(directory, filename))

In [14]:
def download_files_by_date(start_date: str, end_date:str):
    # start_dt = '20221001'
    # end_dt = '20221231'

    # generate dates between a date range
    dates = generate_dates_in_start_end_date(start_date, end_date)


    files_downloaded = []

    for j in range(len(dates)):
        year = dates[j][:4]
        
        for i in range(24):
            url = f'https://s3-us-west-1.amazonaws.com/files.airnowtech.org/airnow/{year}/{dates[j]}/HourlyData_{dates[j]}{get_zero_appended(i)}.dat'
            directory = './data/raw'
            filename = f'HourlyData_{dates[j]}{get_zero_appended(i)}.dat'
            
            files_downloaded.append(f"{directory}{filename}")
            
            t = threading.Thread(target=download_file, args=(url, directory, filename))
            t.start()
    

In [ ]:
# Downloading for April till-date
download_files_by_date('20230101', '20230331')

Exception in thread Thread-1339:
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.8/urllib/request.py", line 1354, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "/opt/homebrew/anaconda3/lib/python3.8/http/client.py", line 1255, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "/opt/homebrew/anaconda3/lib/python3.8/http/client.py", line 1301, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "/opt/homebrew/anaconda3/lib/python3.8/http/client.py", line 1250, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "/opt/homebrew/anaconda3/lib/python3.8/http/client.py", line 1010, in _send_output
    self.send(msg)
  File "/opt/homebrew/anaconda3/lib/python3.8/http/client.py", line 950, in send
    self.connect()
  File "/opt/homebrew/anaconda3/lib/python3.8/http/client.py", line 1424, in connect
    self.sock = self._context.

    with contextlib.closing(urlopen(url, data)) as fp:
  File "/opt/homebrew/anaconda3/lib/python3.8/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/opt/homebrew/anaconda3/lib/python3.8/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/opt/homebrew/anaconda3/lib/python3.8/urllib/request.py", line 542, in _open
    result = self._call_chain(self.handle_open, protocol, protocol +
  File "/opt/homebrew/anaconda3/lib/python3.8/urllib/request.py", line 502, in _call_chain
    result = func(*args)
  File "/opt/homebrew/anaconda3/lib/python3.8/urllib/request.py", line 1397, in https_open
    return self.do_open(http.client.HTTPSConnection, req,
  File "/opt/homebrew/anaconda3/lib/python3.8/urllib/request.py", line 1357, in do_open
    raise URLError(err)
urllib.error.URLError: <urlopen error TLS/SSL connection has been closed (EOF) (_ssl.c:1125)>
Exception in thread Thread-1860:
Traceback (most recent call last):
 

In [8]:

# Directory containing CSV files to be loaded
directory = './data/raw'

# List of CSV files in the directory
csv_files = [os.path.join(directory, file) for file in os.listdir(directory) if file.endswith('.dat')]
len(csv_files)

2035

In [9]:
column_names = ['date', 'hour', 'AQSID', 'sitename', 'GMT offset', 'parameter name', 'reporting units', 'value', 'datasource']


dfs = [pd.read_csv(file, sep='|', names=column_names) for file in csv_files]

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
combined_df

,date,hour,AQSID,sitename,GMT offset,parameter name,reporting units,value,datasource
0,04/21/23,09:00,000020104,CHARLOTTETOWN,-4.0,NO,PPB,0.2,Canada-Prince Edward Island1
1,04/21/23,09:00,000020104,CHARLOTTETOWN,-4.0,NO2,PPB,0.9,Canada-Prince Edward Island1
2,04/21/23,09:00,000020104,CHARLOTTETOWN,-4.0,OZONE,PPB,27.0,Canada-Prince Edward Island1
3,04/21/23,09:00,000020104,CHARLOTTETOWN,-4.0,PM2.5,UG/M3,5.3,Canada-Prince Edward Island1
4,04/21/23,09:00,000020104,CHARLOTTETOWN,-4.0,SO2,PPB,0.0,Canada-Prince Edward Island1
...,...,...,...,...,...,...,...,...,...
12965756,04/21/23,10:00,TT9209004,Miami,-6.0,PM2.5,UG/M3,9.2,Quapaw Nation
12965757,04/21/23,10:00,TT9209004,Miami,-6.0,RHUM,PERCENT,92.6,Quapaw Nation
12965758,04/21/23,10:00,TT9209004,Miami,-6.0,WD,DEGREES,347.0,Quapaw Nation
12965759,04/21/23,10:00,TT9209004,Miami,-6.0,WS,M/S,0.7,Quapaw Nation


In [16]:
def data_segregation(df):
    #%%
    # read file
    # filename = '/Users/varshahindupur/Desktop/GitHub/Aircast/airflow/models/data/AirQualityData.csv'
    # df =  pd.read_csv(filename, sep=',')
    # df.head(2)
    #%%
    # Convert date and hour columns to datetime and combine them
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['hour'], format='%m/%d/%y %H:%M')
    # df.head(2)
    #%%
    # AQSID, aquid, datetime, ozone, no, no2, co, pm2_5, pm10
    pivot_df = df.pivot_table(index=['AQSID', 'datetime'], columns='parameter name', values='value')
    pivot_df = pivot_df.reset_index()
    # Rename the columns to match the desired output format
    pivot_df.columns = ['aquid', 'collection_timestamp', 'AG', 'AL', 'AS', 'AT', 'CH', 'co', 'CR', 'CU', 'FE', 'HG', 'K', 'MN', 'NA', 'NI', 'no', 'no2', 'ozone', 'P', 'PB', 'pm10', 'pm2_5', 'SB', 'ZN']
    # Sort the columns in the desired order
    pivot_df = pivot_df[['aquid', 'collection_timestamp', 'ozone', 'no', 'no2', 'co', 'pm2_5', 'pm10']]
    pivot_df = pivot_df.sort_values(by='collection_timestamp', ascending=False)
    pivot_df.head(15)
    #%%
    # replace NaN values with NULL
    pivot_df = pivot_df.fillna('NULL')
    print(pivot_df)
    #%%
    # pivot_df.to_csv('/Users/varshahindupur/Desktop/GitHub/Aircast/airflow/models/data/AirQualityData_Modified.csv', sep=',', index=False)

In [17]:
new_df = combined_df[0:100000]
# new_df
data_segregation(df= new_df)

           aquid collection_timestamp ozone    no   no2    co pm2_5  pm10
25891  CC0010501  2023-04-21 21:30:00  NULL  NULL  NULL   2.1  NULL  NULL
25867  CC0010102  2023-04-21 21:30:00  NULL  NULL  NULL   1.8  NULL  NULL
26509  CMB010001  2023-04-21 21:30:00  NULL  NULL  NULL  NULL  NULL  NULL
25923  CC0010901  2023-04-21 21:30:00  NULL  NULL  NULL  -0.1  NULL  NULL
26557  DS1010001  2023-04-21 21:30:00  NULL  NULL  NULL  NULL  NULL  NULL
...          ...                  ...   ...   ...   ...   ...   ...   ...
5789   060871005  2023-01-01 01:00:00  NULL  NULL  NULL  NULL  NULL  10.1
1293   000101003  2023-01-01 01:00:00  NULL  NULL  NULL   6.0  NULL  NULL
22707  560330004  2023-01-01 01:00:00  NULL  84.3  NULL  NULL  NULL  -0.6
13470  271095008  2023-01-01 01:00:00  NULL  NULL  NULL  NULL  NULL  NULL
0      000010601  2023-01-01 01:00:00  NULL  NULL  NULL  NULL  NULL  NULL

[26987 rows x 8 columns]


/var/folders/yf/hyl3lym16d16wmscv0hrxjrw0000gn/T/ipykernel_4733/3191180798.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['hour'], format='%m/%d/%y %H:%M')


In [19]:
new_df[new_df['AQSID']=='000010601']

,date,hour,AQSID,sitename,GMT offset,parameter name,reporting units,value,datasource,datetime
15445,04/14/23,14:00,000010601,Goose Bay,-4.0,OZONE,PPB,49.0,Canadian Air and Precipitation Monitoring Network,2023-04-14 14:00:00
23182,01/12/23,23:00,000010601,Goose Bay,-4.0,OZONE,PPB,36.0,Canadian Air and Precipitation Monitoring Network,2023-01-12 23:00:00
30340,01/01/23,15:00,000010601,Goose Bay,-4.0,OZONE,PPB,28.0,Canadian Air and Precipitation Monitoring Network,2023-01-01 15:00:00
45413,01/27/23,02:00,000010601,Goose Bay,-4.0,OZONE,PPB,39.0,Canadian Air and Precipitation Monitoring Network,2023-01-27 02:00:00
52663,02/28/23,19:00,000010601,Goose Bay,-4.0,OZONE,PPB,44.0,Canadian Air and Precipitation Monitoring Network,2023-02-28 19:00:00
56563,01/27/23,16:00,000010601,Goose Bay,-4.0,OZONE,PPB,40.0,Canadian Air and Precipitation Monitoring Network,2023-01-27 16:00:00
71480,01/01/23,01:00,000010601,Goose Bay,-4.0,OZONE,PPB,19.0,Canadian Air and Precipitation Monitoring Network,2023-01-01 01:00:00
86507,02/13/23,23:00,000010601,Goose Bay,-4.0,OZONE,PPB,33.0,Canadian Air and Precipitation Monitoring Network,2023-02-13 23:00:00
97230,01/29/23,19:00,000010601,Goose Bay,-4.0,OZONE,PPB,37.0,Canadian Air and Precipitation Monitoring Network,2023-01-29 19:00:00
